In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df=pd.read_csv('C:/pypy/Data_for_UCI_named.csv')

In [3]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [4]:
df.drop('stab', axis=1,inplace=True)

In [5]:
df.stabf.value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [6]:
df.isnull().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stabf    0
dtype: int64

In [7]:
X=df.drop('stabf',axis=1)
y=df['stabf']

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=1)

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_train_scaled=pd.DataFrame(X_train_scaled,columns=X_train.columns)

X_test_scaled=scaler.transform(X_test)
X_test_scaled=pd.DataFrame(X_test_scaled,columns=X_test.columns)

In [9]:
#Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=1)
rfc.fit(X_train_scaled,y_train)

RandomForestClassifier(random_state=1)

In [10]:
#ExtraTreesClassifier

from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(random_state=1)
etc.fit(X_train_scaled, y_train)

ExtraTreesClassifier(random_state=1)

In [11]:
#XGBoost Classifier

import xgboost
from xgboost import XGBClassifier
xgb=XGBClassifier(random_state=1)
xgb.fit(X_train_scaled, y_train)

C:\Users\Jane Ijeoma\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Users\Jane Ijeoma\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Jane Ijeoma\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, p

[20:16:47] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
#lightGBM

from lightgbm import LGBMClassifier
lgc=LGBMClassifier(random_state=1)
lgc.fit(X_train_scaled, y_train)

LGBMClassifier(random_state=1)

In [38]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score,confusion_matrix

rfc_pred=rfc.predict(X_test_scaled)

In [42]:
# accuracy random forest
rfc_accuracy=accuracy_score(y_true=y_test,y_pred=rfc_pred)
round(rfc_accuracy,4)

0.929

In [43]:
# accuracy xgboost
xgb_pred=xgb.predict(X_test_scaled)
xgb_accuracy=accuracy_score(y_true=y_test,y_pred=xgb_pred)
round(xgb_accuracy,4)

0.9455

In [44]:
# accuracy lgbm
lgc_pred=lgc.predict(X_test_scaled)
lgc_accuracy=accuracy_score(y_true=y_test,y_pred=lgc_pred)
round(lgc_accuracy,4)

0.9395

In [53]:
from sklearn.model_selection import RandomizedSearchCV

params= {'n_estimators': [20,50,200,400,1000],'min_samples_split': [2,3,5,7,9],'min_samples_leaf':[1,2,4,6,8],'max_features': ['auto', 'sqrt', 'log2', None] }
model=RandomizedSearchCV(estimator= etc, cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1,verbose = 1,random_state= 1, param_distributions=params)
model.fit(X_train_scaled,y_train)
model.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\Jane Ijeoma\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Jane Ijeoma\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Jane Ijeoma\AppData\Roaming\Python\Python39\site-packages\sklearn\ensemble\_forest.py", line 476, in fit
    trees = Parallel(
  File "C:\Users\Jane Ijeoma\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispa

{'n_estimators': 100,
 'min_samples_split': 5,
 'min_samples_leaf': 10,
 'max_features': None}

In [59]:
hyper_etc= ExtraTreesClassifier(n_estimators= 100,max_features = None,min_samples_leaf= 10,min_samples_split= 5,random_state = 1)
hyper_etc.fit(X_train_scaled,y_train)
hyper_pred=hyper_etc.predict(X_test_scaled)
acc=accuracy_score(y_test,hyper_pred)
acc

0.9265

In [60]:
etc_pred=etc.predict(X_test_scaled)
acc1=accuracy_score(y_test,etc_pred)
acc1

0.928

In [61]:
etc.feature_importances_

array([0.11739736, 0.11844468, 0.11316851, 0.11546569, 0.03950675,
       0.04037132, 0.04070628, 0.04057864, 0.08978291, 0.09367636,
       0.09688268, 0.09401882])